# MAXP 2021初赛数据探索和处理-1

由于节点的Feature维度比较高，所以先处理节点的ID，并保存。Feature的处理放到第2部分。

In [1]:
import pandas as pd
import numpy as np
import os
import gc

In [2]:
# path
base_path = './dataset'
publish_path = ''

link_p1_path = os.path.join(base_path, publish_path, 'link_phase1.csv')
train_nodes_path = os.path.join(base_path, publish_path, 'train_nodes.csv')
val_nodes_path = os.path.join(base_path, publish_path, 'validation_nodes.csv')
# 假设复赛加入 link_phase2.csv 和 test_nodes.csv
# link_phase2.csv里面包含test中节点（新的节点）和原先train节点的边关系
# test_nodes.csv里面包含id和feature
link_p2_path = os.path.join(base_path, publish_path, 'link_phase2.csv')
test_nodes_path = os.path.join(base_path, publish_path, 'test_nodes.csv')

In [3]:
train_nodes_path

'./dataset/train_nodes.csv'

### 读取边列表并统计节点数量

In [4]:
edge_df = pd.read_csv(link_p2_path)
print(edge_df.shape)
edge_df.head()

(48159841, 3)


,paper_id,reference_paper_id,phase
0,65679d473736e72a8f12984d4c42e955,05f085e5ba961ce715912b6cb4836e96,phase2
1,c90b0164d72bd4f00b4c5efb7c216dc5,33c1a5ee63412996aed098f0a8f01c53,phase2
2,d5945e460d2d63118c6b601c7330e649,50603e852362bde38fa27ea2c9b7039b,phase2
3,f10da75ad1eaf16eb2ffe0d85b76b332,711ef25bdb2c2421c0131af77b3ede1d,phase1
4,9ac5a4327bd4f3dcb424c93ca9b84087,2d91c73304c5e8a94a0e5b4956093f71,phase1


In [ ]:
# phase2
# edge_df2 = pd.read_csv(link_p2_path)
# print(edge_df2.shape)
# edge_df2.head()

In [ ]:
# 将phase1和phase2合并组成全图的新变列表
# edge_df = pd.concat([edge_df, edge_df2])

In [5]:
edge_df.phase.describe()

count     48159841
unique           2
top         phase1
freq      29168650
Name: phase, dtype: object

In [6]:
nodes = pd.concat([edge_df['paper_id'], edge_df['reference_paper_id']])
nodes = pd.DataFrame(nodes.drop_duplicates())
nodes.rename(columns={0:'paper_id'}, inplace=True)

print(nodes.shape)
nodes.head(4)

(4372978, 1)


,paper_id
0,65679d473736e72a8f12984d4c42e955
1,c90b0164d72bd4f00b4c5efb7c216dc5
2,d5945e460d2d63118c6b601c7330e649
3,f10da75ad1eaf16eb2ffe0d85b76b332


#### 在边列表，一共出现了 3,031,367 (nodes.shape[0]) 个节点(paper_id)

## 读取并查看train_nodes和validation_nodes里面的节点

In [7]:
def process_node(line):
    nid, feat_json, label = line.strip().split('\"')
    
    feat_list = [float(feat[1:-1]) for feat in feat_json[1:-1].split(', ')]
    
    if len(feat_list) != 300:
        print('此行数据有问题 {}'.format(line))
    
    return nid[:-1], feat_list, label[1:]

In [8]:
# 先构建ID和Label的关系，保证ID的顺序和Feature的顺序一致即可
nid_list = []
label_list = []
tr_val_list = []

with open(train_nodes_path, 'r') as f:
    i = 0
    
    for line in f:
        if i > 0:
            nid, _, label = process_node(line)
            nid_list.append(nid)
            label_list.append(label)
            tr_val_list.append(0)             # 0表示train的点
        i += 1
        if i % 100000 == 0:
            print('Processed {} train rows'.format(i))

with open(val_nodes_path, 'r') as f:
    i = 0
    
    for line in f:
        if i > 0:
            nid, _, label = process_node(line)
            nid_list.append(nid)
            label_list.append(label)
            tr_val_list.append(1)             # 1表示validation/test的点
        i += 1
        if i % 100000 == 0:
            print('Processed {} validation rows'.format(i))

# test 和 valid 都属于要预测的节点，都用1来表示
with open(test_nodes_path, 'r') as f:
    i = 0
    
    for line in f:
        if i > 0:
            nid, _, label = process_node(line)
            nid_list.append(nid)
            label_list.append(label)
            tr_val_list.append(1)             # 1表示validation/test的点
        i += 1
        if i % 100000 == 0:
            print('Processed {} validation rows'.format(i))
            
nid_arr = np.array(nid_list)
label_arr = np.array(label_list)
tr_val_arr = np.array(tr_val_list)
    
nid_label_df = pd.DataFrame({'paper_id':nid_arr, 'Label': label_arr, 'Split_ID':tr_val_arr})

Processed 100000 train rows
Processed 200000 train rows
Processed 300000 train rows
Processed 400000 train rows
Processed 500000 train rows
Processed 600000 train rows
Processed 700000 train rows
Processed 800000 train rows
Processed 900000 train rows
Processed 1000000 train rows
Processed 1100000 train rows
Processed 1200000 train rows
Processed 1300000 train rows
Processed 1400000 train rows
Processed 1500000 train rows
Processed 1600000 train rows
Processed 1700000 train rows
Processed 1800000 train rows
Processed 1900000 train rows
Processed 2000000 train rows
Processed 2100000 train rows
Processed 2200000 train rows
Processed 2300000 train rows
Processed 2400000 train rows
Processed 2500000 train rows
Processed 2600000 train rows
Processed 2700000 train rows
Processed 2800000 train rows
Processed 2900000 train rows
Processed 3000000 train rows
Processed 100000 validation rows
Processed 200000 validation rows
Processed 300000 validation rows
Processed 400000 validation rows
Process

In [9]:
nid_label_df.reset_index(inplace=True)
nid_label_df.rename(columns={'index':'node_idx'}, inplace=True)
print(nid_label_df.shape)
nid_label_df.head(4)

(5345753, 4)


,node_idx,paper_id,Label,Split_ID
0,0,bfdee5ab86ef5e68da974d48a138c28e,S,0
1,1,78f43b8b62f040347fec0be44e5f08bd,,0
2,2,a971601a0286d2701aa5cde46e63a9fd,G,0
3,3,ac4b88a72146bae66cedfd1c13e1552d,,0


In [10]:
# 检查ID在Train和Validation是否有重复
ids = nid_label_df.paper_id.drop_duplicates()
ids.shape

(5345753,)

#### train和validation一共有3,655,033 (ids.shape[0]) 个节点

#### 下面交叉比对边列表里的paper id和节点列表里的ID，检查是否有匹配不上的节点

In [11]:
inboth = nid_label_df.merge(nodes, on='paper_id', how='inner')
print(inboth.shape)

(4372554, 4)


In [12]:
edge_node = nodes.merge(nid_label_df, on='paper_id', how='left')
print(edge_node.shape)
print('共有{}边列表的节点在给出的节点列表里没有对应，缺乏特征'.format(edge_node[edge_node.node_idx.isna()].shape[0]))
edge_node[edge_node.node_idx.isna()].head(4)

(4372978, 4)
共有424边列表的节点在给出的节点列表里没有对应，缺乏特征


,paper_id,node_idx,Label,Split_ID
1911,cc388eaec8838ce383d8a8792014fedb,NaN,NaN,NaN
2022,5d899f41e52f751fef843cf7b1d05b4a,NaN,NaN,NaN
23867,2b2004ec3c99a44b5cb6045ca547453e,NaN,NaN,NaN
26295,d657c4451a9617f4eec96d3b2e6092c7,NaN,NaN,NaN


#### 合并边列表里独特的节点和train和validation的节点到一起，构成全部节点列表

In [13]:
# 获取未能匹配上的节点，并构建新的节点DataFrame，然后和原有的Train/Validation节点Concat起来
diff_nodes = edge_node[edge_node.node_idx.isna()]
diff_nodes.ID = diff_nodes.paper_id
diff_nodes.Split_ID = 1
diff_nodes.node_idx = 0
diff_nodes.reset_index(inplace=True)
diff_nodes.drop(['index'], axis=1, inplace=True)
diff_nodes.node_idx = diff_nodes.node_idx + diff_nodes.index + 5345753 # 655033  # 3655033需要换成ids.shape[0]的值
diff_nodes = diff_nodes[['node_idx', 'paper_id', 'Label', 'Split_ID']]
diff_nodes.head(4)

/home/chx/anaconda3/envs/env_pyg2/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until
/home/chx/anaconda3/envs/env_pyg2/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/chx/anaconda3/envs/env_pyg2/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,node_idx,paper_id,Label,Split_ID
0,5345753,cc388eaec8838ce383d8a8792014fedb,NaN,1
1,5345754,5d899f41e52f751fef843cf7b1d05b4a,NaN,1
2,5345755,2b2004ec3c99a44b5cb6045ca547453e,NaN,1
3,5345756,d657c4451a9617f4eec96d3b2e6092c7,NaN,1


In [14]:
# Concatenate这424个未匹配到的节点到总的node的最后，从而让nid能接上
nid_label_df = pd.concat([nid_label_df, diff_nodes])
nid_label_df.tail(4)

,node_idx,paper_id,Label,Split_ID
420,5346173,caed47d55d1e193ecb1fa97a415c13dd,NaN,1
421,5346174,c82eb6be79a245392fb626b9a7e1f246,NaN,1
422,5346175,926a31f6b378575204aae30b5dfa6dd3,NaN,1
423,5346176,bbace2419c3f827158ea4602f3eb35fa,NaN,1


In [15]:
# 保存ID和Label到本地文件
nid_label_df.to_csv(os.path.join(base_path, publish_path, './IDandLabels.csv'), index=False)
# 保存未匹配上的节点用于feature的处理
diff_nodes.to_csv(os.path.join(base_path, publish_path, './diff_nodes.csv'), index=False)
